In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from xgboost import XGBClassifier

import joblib

import warnings
warnings.filterwarnings("ignore")

## Load Data

In [36]:
data_df = pd.read_csv('titanic_data.csv')
pipe = joblib.load(f'titanic_pipe.pkl')
pipe.fit(data_df)

data_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Family
0,1,0,3,1,0,22,1,0,7.2500,6,0,1
1,2,1,1,3,1,38,1,0,71.2833,3,1,1
2,3,1,3,2,1,26,0,0,7.9250,6,0,0
3,4,1,1,3,1,35,1,0,53.1000,3,0,1
4,5,0,3,1,0,35,0,0,8.0500,5,0,0


In [37]:
titanic_X = data_df.copy()

del titanic_X['PassengerId']
del titanic_X['Survived']

titanic_Y = np.array(data_df[['Survived']])

x_train, x_test, y_train, y_test = \
   model_selection.train_test_split(titanic_X, titanic_Y, test_size=0.3, random_state=0)

x_train = pipe.transform(x_train)
x_test = pipe.transform(x_test)

In [38]:
clf = XGBClassifier(seed = 1, n_estimators = 300, max_depth = 2)
clf.fit(x_train, y_train)
print(f'XG Boost: ', accuracy_score(clf.predict(x_test), y_test))

XG Boost:  0.832089552238806


In [42]:
params = {'booster': ['gbtree','gblinear','dart'],
          'validate_parameters': [True, False],
          'n_estimators': list(range(100,1000,100)),
          'n_jobs': [-1],
          'random_state': [0],
          'max_depth': list(range(1,9)),
          'learning_rate': [0.01,0.1,0.2,0.3,0.5,0.9],
          'gamma': [0.5, 1, 1.5, 2, 5],
          'subsample': [0.6, 0.8, 1.0],
          'colsample_bytree': [0.6, 0.8, 1.0]}

grid = GridSearchCV(
    XGBClassifier(), params, refit=True, cv=10, scoring='accuracy')

grid.fit(x_train, y_train)

print('GridSearch Best Params:', grid.best_params_)
print(f'Logistic Regression Score:', accuracy_score(y_test, grid.predict(x_test)))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/zq/5f2h2lwd7vd8py_tdcdkh4lm0000gn/T/ipykernel_23706/2475471266.py", line 15, in <cell line: 15>
    grid.fit(x_train, y_train)
  File "/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_search.py", line 891, in fit
    self._run_search(evaluate_candidates)
  File "/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_search.py", line 1392, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_search.py", line 838, in evaluate_candidates
    out = parallel(
  File "/opt/homebrew/lib/python3.9/site-packages/joblib/parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(iterator):
  File "/opt/homebrew/lib/python3.9/site-packages/joblib/para

In [ ]:
folds = 3
param_comb = 5

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X,Y), verbose=3, random_state=1001 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X, Y)
timer(start_time) # timing ends here for "start_time" variable
